# Day 9: Rope Bridge
https://adventofcode.com/2022/day/9

In [2]:
let lines: Vec<String> = std::fs::read_to_string("input/day09.txt")
    .expect("Could not read file")
    .lines()
    .map(|line| line.to_owned())
    .collect();

In [3]:
let example: Vec<String> = "R 4
U 4
L 3
D 1
R 4
D 1
L 5
R 2".lines().map(|line| line.to_owned()).collect();

In [4]:
#[derive(Debug, Clone)]
enum Move {
    Left,
    Right,
    Up,
    Down
}

use Move::*;

In [5]:
impl Move {
    fn parse(direction: &str) -> Move {
        match direction {
            "L" => Left,
            "R" => Right,
            "U" => Up,
            "D" => Down,
            _ => panic!("Could not parse direction: {}", direction)
        }
    }
    
    fn apply(self: &Self, (x, y): (i32, i32)) -> (i32, i32) {
        match self {
            Left => (x - 1, y),
            Right => (x + 1, y),
            Up => (x, y - 1),
            Down => (x, y + 1)
        }
    }
}

In [6]:
:dep itertools = "0.10.5"

In [7]:
use itertools::Itertools;

In [8]:
use itertools::repeat_n;

fn parse_moves(moves: &[String]) -> Vec<Move> {
    moves.iter().flat_map(|m| {
        let mut it = m.split_whitespace();
        let direction = it.next().unwrap();
        let count = it.next().unwrap().parse::<usize>().unwrap();
        assert_eq!(it.next(), None);
        
        repeat_n(Move::parse(&direction), count)
    }).collect()
}

In [9]:
fn apply_moves(moves: Vec<Move>) -> Vec<(i32, i32)> {
    let mut pos = (0, 0);
    moves.iter().map(|m| {
        pos = m.apply(pos);
        pos
    }).collect()
}

## Part 1

In [10]:
fn move_towards((tx, ty): &mut (i32, i32), (hx, hy): (i32, i32)) {
    let dx = hx - *tx;
    let dy = hy - *ty;
    
    if dx.abs() > 1 || dy.abs() > 1 {
        if dx != 0 {
            *tx += dx.signum();
        }

        if dy != 0 {
            *ty += dy.signum();
        }
    }
}

In [11]:
fn tail_positions(moves: Vec<Move>) -> Vec<(i32, i32)> {
    let mut head = (0, 0);
    let mut tail = (0, 0);
    
    moves.iter().map(|m| {
        head = m.apply(head);
        move_towards(&mut tail, head);
        
        tail
    }).collect()
}

In [12]:
use std::collections::HashSet;

In [13]:
fn part1(moves: &[String]) -> usize {
    tail_positions(parse_moves(moves))
        .iter()
        .collect::<HashSet<_>>()
        .len()
}

In [14]:
part1(&example)

13

In [15]:
part1(&lines)

6384

## Part 2

In [16]:
fn tail_positions_part2(moves: Vec<Move>, knots: usize) -> Vec<(i32, i32)> {
    let mut rope = Vec::new();
    rope.resize(10, (0, 0));
    
    moves.iter().map(|m| {
        rope[0] = m.apply(rope[0]);
        
        for i in 1..knots {
            let predecessor = rope[i-1];
            move_towards(&mut rope[i], predecessor);
        }
        
        rope.last().unwrap().clone()
    }).collect()
}

In [17]:
fn part2(moves: &[String]) -> usize {
    tail_positions_part2(parse_moves(moves), 10)
        .iter()
        .collect::<HashSet<_>>()
        .len()
}

In [18]:
part2(&example)

1

In [19]:
let example_part2: Vec<String> = "R 5
U 8
L 8
D 3
R 17
D 10
L 25
U 20".lines().map(|line| line.to_owned()).collect();

In [20]:
part2(&example_part2)

36

In [21]:
part2(&lines)

2734